In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import beta, binom
import itertools
from ipywidgets import interact, interactive


import hashlib
%matplotlib inline

sns.set(style="dark")
plt.style.use("ggplot")

import pymc3 as pm

In [2]:
happy = pd.read_csv('World Happiness Report.csv')
life = pd.read_csv('Life Expectancy Data.csv')

In [3]:
happy = happy.rename(columns={"Country Name": "Country"})

In [15]:
merged = life.merge(happy, how = 'inner', on = ['Country', 'Year'])

In [16]:
columns = ['Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling',
       'Life Ladder', 'Log GDP Per Capita', 'Social Support',
       'Healthy Life Expectancy At Birth', 'Freedom To Make Life Choices',
       'Generosity', 'Perceptions Of Corruption', 'Positive Affect',
       'Negative Affect', 'Confidence In National Government']

In [17]:
merged = merged.dropna()

In [20]:
Status_Indicator = merged.Status.map(dict(Developing=0, Developed=1))

In [21]:
merged['Status Indicator'] = Status_Indicator

In [40]:
merged.std()

/tmp/ipykernel_99/264915096.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  merged.std()


Year                                 2.546042e+00
Life expectancy                      9.109753e+00
Adult Mortality                      1.144230e+02
infant deaths                        1.425491e+02
Alcohol                              4.143097e+00
percentage expenditure               2.136270e+03
Hepatitis B                          2.393083e+01
Measles                              6.517357e+03
 BMI                                 2.019537e+01
under-five deaths                    1.937296e+02
Polio                                2.204027e+01
Total expenditure                    2.391737e+00
Diphtheria                           1.963716e+01
 HIV/AIDS                            3.480102e+00
GDP                                  1.409601e+04
Population                           7.515308e+07
 thinness  1-19 years                4.901240e+00
 thinness 5-9 years                  4.937435e+00
Income composition of resources      1.588976e-01
Schooling                            2.947047e+00


OH NOTES
- sounds more like Bayesian GLM rather than Hierarchical
- could use GLMs as a tool
- Bayesian GLM Non-Parametric and Some Parametric Methods
- Predicting Happiness?
- Doesn't make sense to use Bern bc theta is a random variable between 0 and 1
- you're predicting based on the parameters, predicting the probability that a country is developed, but if you make it bern
- Stop here and write up the report
- Not sure if we can do multipple observed

In [ ]:
def betaParams(mean, std):
    var = std ** 2
    
    alpha = ((1 - mu) / var - (1 / mu)) * (mu ** 2)
    beta = alpha * (1 / mu - 1)
    
    return [alpha, beta]

In [41]:
pi = 0.5       # this is a placeholder
sigma = 0.4    # this is a placeholder
mu_gdp = 9.143147
sigma_gdp = 1.105395

In [42]:
with pm.Model() as develop_prediction:
    
    # is country i developed or not?
    y = pm.Bernoulli('y', p=pi)
    
    # mean gdp per group
    mu = pm.Normal('mu', mu=mu_gdp, sigma=sigma_gdp, shape=2)
    
    gdp_mean = mu[y]
    
    # putting the model together
    x = pm.Normal('x', mu=gdp_mean, sigma=sigma, observed=merged['Log GDP Per Capita'])
    
    trace = pm.sample(2000, chains=1, tune=1000, return_inferencedata=True)

Sequential sampling (1 chains in 1 job)
CompoundStep
>BinaryGibbsMetropolis: [y]
>NUTS: [mu]


Sampling 1 chain for 1_000 tune and 2_000 draw iterations (1_000 + 2_000 draws total) took 4 seconds.
There were 133 divergences after tuning. Increase `target_accept` or reparameterize.
Only one chain was sampled, this makes it impossible to run some convergence checks


In [43]:
trace.posterior

<xarray.Dataset>
Dimensions:   (chain: 1, draw: 2000, mu_dim_0: 2)
Coordinates:
  * chain     (chain) int64 0
  * draw      (draw) int64 0 1 2 3 4 5 6 ... 1993 1994 1995 1996 1997 1998 1999
  * mu_dim_0  (mu_dim_0) int64 0 1
Data variables:
    y         (chain, draw) int64 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    mu        (chain, draw, mu_dim_0) float64 8.932 9.166 8.117 ... 7.346 9.124
Attributes:
    created_at:                 2023-05-04T21:35:10.118820
    arviz_version:              0.12.1
    inference_library:          pymc3
    inference_library_version:  3.11.2
    sampling_time:              3.7984731197357178
    tuning_steps:               1000